In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
import holidays # package to know if a date is a holiday
import xgboost

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/msbd5001-fall2020/train.csv", header=0)
df_train

In [ ]:
df_test = pd.read_csv("/kaggle/input/msbd5001-fall2020/test.csv", header=0)
df_test

In [ ]:
df_train.iloc[10000:10300,:].plot(y="speed")

In [ ]:
for date, name in sorted(holidays.HK(years=2017).items()):
    print(date, name)

In [ ]:
for date, name in sorted(holidays.HK(years=2018).items()):
    print(date, name)

In [ ]:
holidays_list_17 = sorted(holidays.HK(years=2017).keys())
holidays_list_18 = sorted(holidays.HK(years=2018).keys())
holidays_list = holidays_list_17 + holidays_list_18
holidays_list

In [ ]:
df_train["date"] = pd.to_datetime(df_train["date"])

In [ ]:
df_train['dateNOTIME'] = pd.to_datetime(df_train['date']).dt.date

In [ ]:
df_train["holiday"] = (df_train["dateNOTIME"].isin(holidays_list)).astype(int)

In [ ]:
df_train["year"] = df_train["date"].apply(lambda x: x.year)

In [ ]:
df_train["month"] = df_train["date"].apply(lambda x: x.month)

In [ ]:
df_train["Day_in_month"] = df_train["date"].apply(lambda x: x.day)

In [ ]:
df_train["day"] = df_train["date"].apply(lambda x: x.weekday())

In [ ]:
df_train["hour"] = df_train["date"].apply(lambda x: x.hour)

In [ ]:
df_train["day_off"] = ((df_train["day"]==6) | (df_train["holiday"]==1)).astype(int)

In [ ]:
df_avg_holidays = pd.DataFrame({})
df_avg_holidays["mean_holiday"] = df_train.groupby("holiday")["speed"].mean()
df_avg_holidays["std_holiday"] = df_train.groupby("holiday")["speed"].std()
df_avg_holidays

In [ ]:
df_avg_holidays_hour = pd.DataFrame({})
df_avg_holidays_hour["mean_hour_holiday"] = df_train.groupby(["holiday", "hour"])["speed"].mean()
df_avg_holidays_hour["std_hour_holiday"] = df_train.groupby(["holiday", "hour"])["speed"].std()
df_avg_holidays_hour

In [ ]:
df_avg_holidays_day_hour = pd.DataFrame({})
df_avg_holidays_day_hour["mean_hour_holiday"] = df_train.groupby(["holiday","day", "hour"])["speed"].mean()
df_avg_holidays_day_hour["std_hour_holiday"] = df_train.groupby(["holiday","day", "hour"])["speed"].std()
df_avg_holidays_day_hour

In [ ]:
df_avg_day = pd.DataFrame({})
df_avg_day["mean_day"] = df_train.groupby("day")["speed"].mean()
df_avg_day["std_day"] = df_train.groupby("day")["speed"].std()
df_avg_day

In [ ]:
df_avg_day.plot(y="mean_day")

In [ ]:
df_avg_hour = pd.DataFrame({})
df_avg_hour["mean_hour"] = df_train.groupby("hour")["speed"].mean()
df_avg_hour["std_hour"] = df_train.groupby("hour")["speed"].std()
df_avg_hour

In [ ]:
df_avg_hour.plot(y="mean_hour")

In [ ]:
df_avg_hour_day = pd.DataFrame({})
df_avg_hour_day["mean_day_hour"] = df_train.groupby(["day","hour"])["speed"].mean()
df_avg_hour_day["std_day_hour"] = df_train.groupby(["day","hour"])["speed"].std()
df_avg_hour_day

In [ ]:
df_avg_hour_day.plot(y="mean_day_hour")

In [ ]:
df_avg_hour_day.plot(y="std_day_hour")

In [ ]:
df_avg_month_day_hour = pd.DataFrame({})
df_avg_month_day_hour["mean_month_day_hour"] = df_train.groupby(["month","day","hour"])["speed"].mean()
df_avg_month_day_hour["std_month_day_hour"] = df_train.groupby(["month","day","hour"])["speed"].std()
df_avg_month_day_hour

In [ ]:
#df_full_date = pd.DataFrame({"date":pd.date_range(start='1/1/2017', end='31/12/2018', freq='H').to_list()})
#df_full_date

In [ ]:
#x_train = df_train[["id", date","year","month", "day", "hour", "holiday"]]
x_train = df_train[["id","year","month", "day", "Day_in_month", "day_off", "hour", "holiday"]]
y_train = df_train[["speed"]]

In [ ]:
# x = x_train.set_index(["month", "day", "hour"])

In [ ]:
"""
x = pd.merge(x, df_avg_month_day_hour[["mean_month_day_hour"]], left_index=True, right_index=True)
x = pd.merge(x, df_avg_hour_day[["mean_day_hour"]], left_index=True, right_index=True)
X_Train = x.reset_index(level=["month", "day", "hour"])
X_Train = X_Train.sort_values(by='date').reset_index(drop=True)
X_Train = X_Train[["id","year","month", "day", "Day_in_month", "hour", "holiday","day_off", "mean_month_day_hour","mean_day_hour"]]
"""

In [ ]:
y_train

In [ ]:

xgb_model = xgboost.XGBRegressor(objective='reg:squarederror', tree_method='gpu_hist')

params = {
    "colsample_bytree": uniform(loc=0.8, scale=0.2),
    "gamma": uniform(loc=0, scale=5),
    "learning_rate": uniform(loc=0.005, scale=0.001), 
    "max_depth": randint(4, 8), 
    "n_estimators": randint(1000, 1001),
    "subsample": uniform(loc=0.8, scale=0.2)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=152, n_iter=4, cv=3, 
                            verbose=1, n_jobs=1, scoring='neg_mean_squared_error', return_train_score=True)

search.fit(x_train, y_train)

search.cv_results_



In [ ]:
results = pd.DataFrame(search.cv_results_)
results

In [ ]:
model = search.best_estimator_

In [ ]:
df_test["date"] = pd.to_datetime(df_test["date"])
df_test['dateNOTIME'] = pd.to_datetime(df_test['date']).dt.date
df_test["holiday"] = (df_test["dateNOTIME"].isin(holidays_list)).astype(int)
df_test["year"] = df_test["date"].apply(lambda x: x.year)
df_test["month"] = df_test["date"].apply(lambda x: x.month)
df_test["Day_in_month"] = df_test["date"].apply(lambda x: x.day)
df_test["day"] = df_test["date"].apply(lambda x: x.weekday())
df_test["hour"] = df_test["date"].apply(lambda x: x.hour)
df_test["day_off"] = ((df_test["day"]==6) | (df_test["holiday"]==1)).astype(int)

In [ ]:
df_test

In [ ]:
x_test = df_test[["id","year","month", "day", "Day_in_month", "day_off", "hour", "holiday"]]
x_test

In [ ]:
predict = model.predict(x_test)

In [ ]:
df_test["speed"] = predict
df_test

In [ ]:
df_submission = df_test[["id", "speed"]]

In [ ]:
df_submission.to_csv("Submission.csv", index=False)